In [1]:
import re
import pandas as pd
import bz2

In [2]:
file_name = "C:/Users/User/Desktop/enwiki-latest-pages-articles10.xml-p4045403p5399366.bz2"

with bz2.open(file_name, "rb") as f:
    content = f.read().decode('utf-8')

In [156]:
def find_section_headers(text, file):
    for match in re.finditer("((={2,})( ?)(.*?)( ?)(={2,}))|<title>(.*)</title>",text):
        x = re.search("\(\{\{[Nn]o [Mm]ore [Ll]inks\}\}\)",match.group())
        if x:
            continue
        if match.group(3) or match.group(5):
            k = match.group(2) + match.group(4) + match.group(6) + "\n"
        else:
            k = match.group() + "\n"
        file.write(k)
    file.close()

In [127]:
def image_in_section(path):
    img = " ?\[\[File:[^\]]+\]\]"
    x = re.search(img,path)
    if x:
        path = re.sub(img,"",path)
    return path

In [128]:
def delete_linking(path):
    #x = re.search("\[\[(.*)?\|(.*)?\]\]|\[\[(.*)?\]\]",path)
    path = image_in_section(path)
    for x in re.finditer("\[\[([^\]]+)\|([^\]]+)\]\]|\[\[([^\]]+)\]\]",path):
        if x:
            if x.group(3):
                path = re.sub(re.escape(x.group()), x.group(3),path)
            elif x.group(2):
                path = re.sub(re.escape(x.group()), x.group(2),path)
   
    return path

In [5]:
def add(x, y):
    z = x + "\t" + y
    return z

In [70]:
def hierarchy(text, file_headers):
    for match in re.finditer("(={2,})(.*?)={2,}|<title>(.*)</title>",text):
        if match.group()[0] == "<":
            title = match.group(3)
            is_wiki_or_file = 0
            if (len(re.findall("Wikipedia:|File:|Help:|Template:",title)) > 0):
                is_wiki_or_file = 1
                continue
            level = 0
            path = title    
        elif (is_wiki_or_file == 0):
            if len(match.group(1)) > level:
                temp = path
                level = len(match.group(1))
                path = add(match.group(0),path)
                new_path = re.sub("=","",path);
                #new_path = image_in_section(new_path)
                new_path = delete_linking(new_path)
                file_headers.write(new_path + "\n")
                #print(path + "\n")

            elif len(match.group(1)) == level:
                path = add(match.group(0),temp)
                new_path = re.sub("=","",path);
                #new_path = image_in_section(new_path)
                new_path = delete_linking(new_path)
                file_headers.write(new_path + "\n")
                #print(path + "\n")

            else:
                while(1):
                    x = re.search("(={2,})(.*?)\t",temp)
                    if x:
                        if len(match.group(1)) <= len(x.group(1)):
                            temp = temp[(x.end()):]
                            if len(match.group(1)) == len(x.group(1)):
                                break
                        else:
                            break
                    else:
                        break

                level = len(match.group(1))
                path = add(match.group(0),temp)
                new_path = re.sub("=","",path);
                #new_path = image_in_section(new_path)
                new_path = delete_linking(new_path)
                file_headers.write(new_path + "\n")

                #print(path + "\n")

    file_headers.close()          

In [166]:
def edit_characters(file_unedit, text):
    x = re.sub("\'\'","",text)
    x = re.sub("&quot;","\"",x)
    x = re.sub("&amp;","&",x)
    x = re.sub("( ?\{\{[Aa]nchor ?\|[^\}]+\}\} ?|\{\{[Aa]nchor\}\})","",x)
    file_unedit.write(x)
    file_unedit.close()

In [158]:
file_parse_bigger_data = open("data/file_parse_bigger_data.txt","w",encoding="utf8")
find_section_headers(content, file_parse_bigger_data)

In [159]:
file = open("data/file_parse_bigger_data.txt", encoding="utf8")
text = file.read()
file.close()

file_headers = open("data/headers_bigger_data.txt","w",encoding="utf8")
hierarchy(text,file_headers)

In [167]:
file_unedited = open("data/headers_bigger_data.txt", encoding="utf8")
text = file_unedited.read()
file_unedited.close()

file_edit = open("data/headers_bigger_data_edited.txt","w",encoding="utf8")
edit_characters(file_edit, text)
